## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [81]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

#sys.path

In [82]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

### Reading Datasets

In [83]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [84]:
df_other_methods['method'].unique()

array(['ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF', 'InceptionTime',
       'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet', 'ProximityForest', 'WEASEL',
       'Catch22', 'cBOSS'], dtype=object)

In [85]:
# Set directory where the csv files are located
#directory = os.getcwd()+'/ResultsRsast'
directory = os.getcwd()+'/results_accuracy_per_ds'
# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,0.944606,0.094786,0.008965,0.023696,0.049890,0.011838,Chinatown,ACF&PACF: n_random_points=1 nb_inst_per_class=...,1,(max instances per class)//2,Rsast,ACF&PACF
1,1,0.927114,0.054925,0.008766,0.007246,0.027613,0.010875,Chinatown,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,(lenthg ts)//2,1,Rsast,ACF&PACF
2,2,0.906706,0.072878,0.016326,0.016857,0.037296,0.002049,Chinatown,ACF: n_random_points=1 nb_inst_per_class=(max ...,1,(max instances per class)//2,Rsast,ACF


In [86]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")
df_result['dataset_name'].unique()


array(['Chinatown', 'SyntheticControl'], dtype=object)

In [87]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [88]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="STC"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('method',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,dataset,score
0,Chinatown,0.901846,0.930434,0.945416,0.937804,0.917153,0.946712,Chinatown,0.967930
1,SyntheticControl,0.690556,0.851667,0.661944,0.877778,0.869444,0.850000,SyntheticControl,0.996667


In [91]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10
len_method,,,
ACF,0.930758,0.792307,0.851251
ACF&PACF,0.933916,0.884679,0.921356
Max ACF,0.960155,0.812500,0.879525
Max PACF,0.942663,0.904630,0.927721
None,0.923226,0.881535,0.916915
PACF,0.971331,0.883892,0.928644


In [92]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10
len_method,,,
ACF,0.930272,0.740633,0.903249
ACF&PACF,0.955782,0.847839,0.943619
Max ACF,0.952381,0.798588,0.898620
Max PACF,0.945335,0.882482,0.948087
None,0.912293,0.851285,0.954454
PACF,0.932216,0.895689,0.942924


#### Time: Subsequence Lenght Method 

In [93]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,Chinatown,SyntheticControl
len_method,,
ACF,0.046705,2.425012
ACF&PACF,0.180019,7.053521
Max ACF,0.046507,0.933822
Max PACF,0.050432,0.876978
None,0.042346,0.665407
PACF,0.075387,5.148298


In [94]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10
len_method,,,
ACF,0.048859,0.189526,1.487986
ACF&PACF,0.071796,0.687580,4.326940
Max ACF,0.047129,0.102382,0.581760
Max PACF,0.046741,0.095529,0.558827
None,0.047039,0.080462,0.416476
PACF,0.073387,0.364531,3.169518


In [95]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10
len_method,,,
ACF,0.042929,0.213882,1.467584
ACF&PACF,0.097133,0.784458,4.213171
Max ACF,0.054164,0.124131,0.555321
Max PACF,0.059882,0.147335,0.498260
None,0.042960,0.095934,0.403723
PACF,0.087016,0.463300,3.061663


#### Variance: Subsequence Lenght Method 

In [73]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,(max instances per class)//2,1,10
len_method,,,,,
ACF,0.018572,0.015314,0.003628,0.007819,0.000049
ACF&PACF,0.015403,0.004760,0.000341,0.003388,0.000022
Max ACF,0.035079,0.012525,0.001496,0.000577,0.000078
Max PACF,0.002485,0.001803,0.002003,0.004562,0.000007
None,0.007226,0.001296,0.008165,0.005768,0.000207
PACF,0.003870,0.003122,0.007533,0.000210,0.000015


In [96]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,(lenthg ts)//2,1,10
len_method,,,
ACF,0.005287,0.024463,0.022776
ACF&PACF,0.003778,0.011558,0.005705
Max ACF,0.000422,0.034220,0.014923
Max PACF,0.003564,0.003118,0.002700
None,0.005040,0.010475,0.001912
PACF,0.000227,0.006351,0.006694


#### Another Results

In [97]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,288,288,288
unique,2,6,48
top,Chinatown,ACF&PACF,ACF&PACF: n_random_points=1 nb_inst_per_class=...
freq,216,48,6


In [98]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
Chinatown,0.004414
SyntheticControl,0.057010


In [99]:
df_result['time'].sum()/(60*60)

0.061424088213178844

#### Generate CD Diagram Tunning Hyperparameter

In [100]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()

In [86]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='images_cd_diagram/cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not 

In [87]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,0
Max PACF: n_random_points=30 nb_inst_per_class=1,127.916667
Max ACF: n_random_points=30 nb_inst_per_class=1,123.250000
Max ACF: n_random_points=1 nb_inst_per_class=1,121.875000
Max ACF: n_random_points=10 nb_inst_per_class=1,117.541667
Max PACF: n_random_points=1 nb_inst_per_class=30,116.666667
...,...
ACF&PACF: n_random_points=100 nb_inst_per_class=50,42.750000
ACF&PACF: n_random_points=100 nb_inst_per_class=30,42.375000
PACF: n_random_points=100 nb_inst_per_class=30,41.375000
ACF&PACF: n_random_points=10 nb_inst_per_class=100,41.125000


In [88]:
#generate example cd diagrams
draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


#### Focus on most accurate lenght method

##### Overall Accuracy

In [89]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points=50 nb_inst_per_class=100'], dtype='object')

In [90]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.716990,0.707127,0.813616,0.809033,0.813090
10,0.736965,0.814283,0.818863,0.814046,0.820416
30,0.767948,0.805414,0.816957,0.816813,0.819020
50,0.807897,0.811014,0.800429,0.821346,0.825309
100,0.800016,0.809959,0.818897,0.818032,0.815656


In [91]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.066533,0.080303,0.030073,0.040424,0.033693
10,0.062014,0.033533,0.039029,0.043808,0.041195
30,0.041493,0.049400,0.038865,0.045356,0.041841
50,0.024748,0.038161,0.060122,0.044564,0.040154
100,0.028426,0.040994,0.043260,0.044824,0.046722


##### Overall time complexity

In [92]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
BirdChicken,0.001775,1.845793,0.000515,3.106501,4.954678
Chinatown,0.000160,0.001823,0.000126,0.005264,0.007402
Coffee,0.001159,0.119599,0.000517,1.587549,1.708893
DistalPhalanxOutlineAgeGroup,0.000686,0.038889,0.010279,6.936574,6.986530
DistalPhalanxOutlineCorrect,0.000761,0.028729,0.012558,8.410944,8.453076
GunPoint,0.000717,0.042117,0.001210,1.425890,1.470008
ItalyPowerDemand,0.000144,0.003278,0.000300,0.030122,0.033879
ShapeletSim,0.002136,2.069743,0.000366,1.631211,3.703531
SmoothSubspace,0.000118,0.003916,0.000706,0.052012,0.056797


In [93]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.059267,0.446878,0.526805,0.518697,0.508688
10,0.079947,0.725623,1.075959,1.264428,1.504154
30,0.137414,1.335132,2.261715,2.975149,3.811814
50,0.211412,2.012466,3.960022,4.480130,6.171286
100,0.355615,3.548282,6.272142,8.778900,11.194438


##### Calculate weights time complexity

In [94]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.037933,0.035040,0.037850,0.038401,0.041002
10,0.040538,0.036245,0.042641,0.043203,0.047068
30,0.044977,0.042565,0.046539,0.050167,0.048851
50,0.044484,0.045780,0.045468,0.048524,0.049981
100,0.050934,0.043582,0.049313,0.049783,0.046869


##### Finding subsequences time complexity

In [95]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,2.619535,24.856164,27.900273,26.070745,23.765545
10,2.788928,25.006459,28.177670,26.608351,24.043635
30,2.542850,24.268097,29.010061,31.668102,30.311706
50,3.031036,30.201905,32.043904,28.308357,27.977514
100,3.025260,28.780339,26.399535,40.387128,29.022799


##### Transform Dataset time complexity

In [96]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.878649,1.860593,3.620241,4.956047,6.655589
10,1.918754,18.417328,36.243088,49.119594,66.034704
30,5.594412,55.710952,106.496284,146.490877,196.670274
50,9.524848,90.367184,178.356234,240.140503,323.483648
100,18.173916,183.906047,348.941621,484.973417,640.888910


##### Classifier time complexity

In [97]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.017345,0.059865,0.048935,0.055273,0.057892
10,0.047579,0.076098,0.092173,0.092033,0.120973
30,0.061133,0.084119,0.145335,0.294360,1.670984
50,0.081386,0.128701,27.149624,0.301612,18.757385
100,0.082500,0.160778,0.927903,1.310779,1.693620
